# 12. Markov chain Monte Carlo

## Brief summary

### Metropolis-Hastings

Let $\boldsymbol{s} = (s_1, ..., s_M)$ be a desired stationary distribution on state space ${1, ..., M}$. Assume that $s_i > 0$ for all $i$ (if not, just delete any states $i$ with $s_i=0$ from the state space). Suppose that $P = (p_{ij})$ is the transition matrix for a Markov chain on state space ${1, ..., M}$. Intuitively, $P$ is a Markov chain that we know how to run but that doesn't have the desired stationary distribution.

Our goal is to modify $P$ to construct a Markov chain $X_0, X_1, ...$ with stationary distribution $\boldsymbol{s}$. We will give a Metropolis-Hastings algorithm for this. Start at any state $X_0$ (chosen randomly or deterministically), and suppose that the new chain is currently at $X_n$. To make one of the new chain, do the following.

1. If $X_n = i$, propose a new state $j$ using the transition probabilities in the $i$th row of the original transition matrix $P$.
2. Compute the *acceptance probability*
\begin{equation}
a_{ij} = min\big( \frac{s_jp_{ji}} {s_ip_{ij}}, 1 \big)
\end{equation}
3. Flip a coin that lands Heads with probability $a_{ij}$.
4. If the coin lands Heads, accept the proposal (i.e., go to $j$), setting $X_{n+1} = j$. Otherwise, reject the proposal (i.e., stay at $i$), setting $X_{n+1}=i$.

- Both $\boldsymbol{s}$ and $P$ are very general, and nothing was stipulated about their being related. In practice, however, the choice of the proposal distribution is extremely important since it can make an enormous difference in how *fast* the chain converges to its stationary distribution.


### Gibbs sampling

#### Systematic scan Gibbs sampler

Let $X$ and $Y$ be discrete r.v.s with joint PMF $p_{X,Y}(x,y) = P(X=x, Y=y)$. We wish to construct a two-dimensional Markov chain $(X_n, Y_n)$ whose stationary distribution is $p_{X,Y}$. The systematic scan Gibbs sampler proceeds by updating the X-component and the Y-component in alternation. If the current state is $(X_n,Y_n)=(x_n,y_n)$, then we update the X-component while holding the Y-component fixed, and then update the Y-component while holding the X-component fixed:

1. Draw a value $x_{n+1}$ from the conditional distribution of $X$ given $Y=y_n$, and set $X_{n+1} = x_{n+1}$.
2. Draw a value $y_{n+1}$ from the conditional distribution of $Y$ given $X=x_{n+1}$, and set $Y_{n+1} = y_{n+1}$.

Repeating steps 1 and 2 over and over, the stationary distribution of the chain $(X_0,Y_0)$, $(X_1,Y_1)$, $(X_2,Y_2)$, $...$ is $p_{X,Y}$.

#### Random scan Gibbs sampler

Let $X$ and $Y$ be discrete r.v.s with joint PMF $p_{X,Y}(x,y) = P(X=x, Y=y)$. We wish to construct a two-dimensional Markov chain $(X_n, Y_n)$ whose stationary distribution is $p_{X,Y}$. Each move of the random scan Gibbs sampler picks a uniformly random component and updates it, according to the conditional distribution given the other component:

1. Choose which component to update, with equal probabilities.
2. If the X-component was chosen, draw a value $x_{n+1}$ from the conditional distribution of $X$ given $Y=y_n$, and set $X_{n+1} = x_{n+1}$. $Y_{n+1}=y_n$. Similarly, if the Y-component was chosen, draw a value $y_{n+1}$ from the conditional distribution of $Y$ given $X=x_n$, and set $X_{n+1}=x_n$, $Y_{n+1} = y_{n+1}$.

Repeating steps 1 and 2 over and over, the stationary distribution of the chain $(X_0,Y_0)$, $(X_1,Y_1)$, $(X_2,Y_2)$, $...$ is $p_{X,Y}$.

Gibbs sampling generalizes naturally to higher dimensions. If we want to sample from a $d$-dimensional joint distribution, the Markov chain we construct will be a sequence of $d$-dimensional random vectors. At each stage, we choose one component of the vector to update, and we draw from the conditional distribution of that component given the most recent values of the other components. We can either cycle through the components of the vector in a systematic order, or choose a random component to update each time.

#### Random scan Gibbs as Metropolis-Hastings

The random scan Gibbs sampler is a special case of the Metropolis-Hastings algorithm, in which the proposal is *always* accepted. In particular, it follows that the stationary distribution of the random scan Gibbs sampler is as desired.


## Python examples

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, binom, beta, poisson
from numpy.random import choice
from numpy.linalg import matrix_power

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Metropolis-Hastings

#### Normal-Normal conjugacy

Let $Y|\theta \sim \mathcal{N}(\theta, \sigma^2)$, where $\sigma^2$ is known but $\theta$ is unknown. Using the Bayesian framework, we treat $\theta$ as a random variable, with prior given by $\theta \sim \mathcal{N}(\mu, \tau^2)$ for some known constants $\mu$ and $\tau^2$. That is, we have the two-level model

\begin{equation}
\theta \sim \mathcal{N}(\mu, \tau^2) \\
Y|\theta \sim \mathcal{N}(\theta, \sigma^2).
\end{equation}

Describe how to use the Metropolis-Hastings algorithm to find the posterior mean and variance of $\theta$ after observing the value of $Y$.

*Solution:*
After observing $Y=y$, we can update our prior uncertainty for $\theta$ using Bayes' rule. Because we are interested in the posterior distribution of $\theta$, any terms not depending on $\theta$ can be treated as part of the normalizing constant. Thus,

\begin{equation}
f_{\theta|Y}(\theta|y) \propto f_{Y|\theta}(y|\theta)f_{\theta}(\theta) 
\propto e^{-\frac{1}{2\sigma^2} (y-\theta)^2} e^{-\frac{1}{2\tau^2} (\theta-\mu)^2}.
\end{equation}

Since we have a quadratic function of $\theta$ in the exponent, we recognize the posterior PDF of $\theta$ as a Normal PDF. The posterior distribution stays in the Normal family, which tells us that *the Normal is the conjugate prior of the Normal*. In fact, by completing the square (a rather tedious calculation which we shall omit), we can obtain an explicit formula for the posterior distribution of $\theta$:

\begin{equation}
\theta|Y=y \sim \mathcal{N}\big( \frac{\frac{1}{\sigma^2}}{\frac{1}{\sigma^2} + \frac{1}{\tau^2}}y + \frac{\frac{1}{\tau^2}}{\frac{1}{\sigma^2} + \frac{1}{\tau^2}}\mu, \frac{1}{\frac{1}{\sigma^2} + \frac{1}{\tau^2}} \big).
\end{equation}

Let's suppose we didn't know how to complete the square, or that we wanted to check our calculations for specific values of $y$, $\sigma^2$, $\mu$, and $\tau^2$. We can do this by simulating from the posterior distribution of $\theta$, using the Metropolis-Hastings algorithm to construct a Markov chain whose stationary distribution is $f_{\theta|Y}(\theta|y)$. A Metropolis-Hastings algorithm for generating $\theta_0, \theta_1, ...$ is as follows.

1. If $\theta_n=x$, propose a new state $x'$ according to some transition rule. One way to do this in a continuous state space is to generate a Normal r.v. $\epsilon_n$ with mean $0$ and add it onto the current state to get the proposed state: in other words, we generate $\epsilon_n \sim \mathcal{N}(0, d^2)$ for some constant $d$, and then set $x' = x + \epsilon_n$. This is the analog of a transition matrix for a continuous state space. The only additional detail is deciding $d$; in practice, we try to choose a moderate value that is neither too large nor too small.
2. The acceptance probability is
\begin{equation}
a(x,x') = min\big(\frac{\boldsymbol{s}(x')p(x',x)} {\boldsymbol{s}(x)p(x,x')}, 1\big),
\end{equation}
where $\boldsymbol{s}$ is the desired stationary PDF (this was a PMF in the discrete case) and $p(x, x')$ is the probability *density* of proposing $x'$ from $x$ (this was $p_{ij}$ in the discrete case).
In this problem, we want the stationary PDF to be $f_{\theta|Y}$, so we'll use that for $\boldsymbol{s}$. As for $p(x,x')$, proposing $x'$ from $x$ is the same as having $\epsilon_n = x'-x$, so we evaluate the PDF of $\epsilon_n$ at $x'-x$ to get
\begin{equation}
p(x,x') = \frac{1}{\sqrt{2\pi}d} e^{-\frac{1}{2d^2}(x'-x)^2A.}
\end{equation}
However, since $p(x',x) = p(x,x')$, these terms cancel from the acceptance probability, leaving us with
\begin{equation}
a(x,x') = min\big( \frac{f_{\theta|Y}(x'|y)}{f_{\theta|Y}(x|y)}, 1 \big).
\end{equation}
3. Flip a coin that lands Heads with probability $a(x,x')$, independently of the Markov chain.
4. If the coin lands Heads, accept the proposal and set $\theta_{n+1}=x'$. Otherwise, stay in place and set $\theta_{n+1}=x$.

In [ ]:
# Choose our observed value of Y 
# and decide on values for the constants \theta, \mu, and \tau:
y = 3
sigma = 1
mu = 0 
tau = 2
d = 1
niter = 10**4
Theta = np.zeros(niter, dtype=np.float)   # allocate a vector Theta of length niter

In [ ]:
# Initialize \theta to the observed value y, then run the algorithm:
Theta[0] = y
for i in range(1, niter):
    theta_new = Theta[i-1] + norm.rvs(scale=d, size=1)
    # Compute the acceptance probability:
    r = norm.pdf(y, loc=theta_new, scale=sigma) * norm.pdf(theta_new, loc=mu, scale=tau) / \
         (norm.pdf(y, loc=Theta[i-1], scale=sigma) * norm.pdf(Theta[i-1], loc=mu, scale=tau))
    flip = binom.rvs(n=1, p=min(r, 1), size=1)
    Theta[i] = theta_new if flip == 1 else Theta[i-1]

In [ ]:
# Discard some of the initial draws 
# to give the chain some time to approach the stationary distribution:
Theta_latter = Theta[niter/2:]

In [ ]:
# Create a histogram
plt.figure(figsize=(10, 5))
_ = plt.hist(Theta_latter, bins=100)

### Gibbs

#### Chicken-egg with unknown parameters

A chicken lays $N$ eggs, where $N \sim Pois(\lambda)$. Each egg hatches with probability $p$, where $p$ is unknown; we let $p \sim Beta(a,b)$. The constants $\lambda, a, b$ are known.

We don't get to observe $N$. Instead, we only observe the number of eggs that hatch, $X$. Describe how to use Gibbs sampling to find $E(p|X=x)$, the posterior mean of $p$ after observing $x$ hatched eggs.

*Solution:*

By the chicken-egg story, the distribution of $X$ given $p$ is $Pois(\lambda p)$. The posterior PDF of $p$ is proportional to

\begin{equation}
f(p|X=x) \propto P(X=x|p)f(p) \propto e^{-\lambda p}(\lambda p)^xp^{a-1}q^{b-1},
\end{equation}

where we have dropped all terms not depending on $p$.

Conditional on observing $N=n$ and knowing the true value of $p$, the distribution of $X$ would be $Bin(n,p)$. By conditioning on the total number of eggs, we recover *Beta-Binomial conjugacy* between $p$ and $X$:

\begin{equation}
p|X=x, N=n \sim Beta(x+a, n-x+b).
\end{equation}

We make an initial guess for $p$ and $N$, then iterate the following steps:

1. Conditional on $N=n$ and $X=x$, draw a new guess for $p$ from the $Beta(x+a, n-x+b)$ distribution.
2. Conditional on $p$ and $X=x$, the number of unhatched eggs is $Y \sim Pois(\lambda(1-p))$ by the chicken-egg story, so we can draw $Y$ from the $Pois(\lambda(1-p))$ distribution and set the new guess for $N$ to be $N=x+Y$.

In [ ]:
# Decide the observed value of X, as well as the constants \lambda, a, b:
x = 7
l = 10
a = 1
b = 1
niter = 10**4
P = np.zeros(niter, dtype=np.float)
N = np.zeros(niter, dtype=np.int)

In [ ]:
# Initialize p and N to the values 0.5 and 2x, respectively,
# then run the algorithm:
P[0] = 0.5
N[0] = 2*x
for i in range(1, niter):
    P[i] = beta.rvs(a=x+a, b=N[i-1]-x+b)
    N[i] = x + poisson.rvs(mu=l*(1-P[i-1]))

In [ ]:
# Discard some of the initial draws 
# to give the chain some time to approach the stationary distribution:
P_latter = P[niter/2:]
N_latter = N[niter/2:]

In [ ]:
# Create a histogram
plt.figure(figsize=(10, 10))
fig, axes = plt.subplots(2, 1)
_ = axes[0].hist(P_latter, bins=100)
_ = axes[1].hist(N_latter, bins=100)